# Swarming

In nature, swarms of animals are capable of amazing feats. For example schools of fish are able to confuse and evade a predator, swarms of ants are able to find the shortest path towards a food source, and swarms of honeybees can make informed choices on new nest locations. It is inspiring to see how animals with limited cognitive abilities are able to transcend their individual capabilities by being part of a swarm. 

The field of _swarm robotics_ [1,2] has two main goals:
<OL>
    <LI style="margin-bottom: 10px;"><B>Understand biological systems:</B> The robots (or simulated agents) in the swarm serve as a model for animals. This allows to test hypotheses on the animals and their governing behavioral rules.</LI>
    <LI style="margin-bottom: 10px;"><B>Design robotic swarms:</B> This goal entails setting up a system consisting of many robots that are individually limited, but that together are able to perform complex tasks. Robotic swarms promise to form a _flexible_, _robust_, and _scalable_ solution.</LI>
</OL>

A central challenge in swarming research is that it concerns a _complex_ system in the true sense of the word:

_"**Complexity** characterises the behaviour of a system or model whose components interact in multiple ways and follow local rules, meaning there is no reasonable higher instruction to define the various possible interactions" -_ <A HREF="https://en.wikipedia.org/wiki/Complexity" TARGET="_blank">Wikipedia</A>

In order to get a better understanding of the issues involved in swarm robotics, this notebook will have you implement some basic behaviors for a swarm of simulated drones.         

## Simple swarm environment and agents

We will employ a simple, rectangular environment with simple agents that have a position, heading and velocity. The agent can determine velocity and heading rate commands, which are executed with a certain delay (low-pass filtered). All classes, `Environment`, `Agent`, etc. are defined in the file `swarming.py` (available <A HREF="https://raw.githubusercontent.com/guidoAI/swarming_notebook/main/swarming.py" TARGET="_blank">here</A>).

Let us first make and run 10 "random" agents. These agents draw their velocity $v$ and heading rate $r$ from a uniform random distribution, and are hence blissfully unaware of each other. Specifically, $v \in [0, 1]$ and $r \in [-0.1 \pi, 0.1 \pi]$.

<FONT COLOR="red"><B>Exercise 1</B></FONT>
<OL>
    <LI>**Before running the code and reading on:** What do you expect the random agents to do? Do you expect a global pattern to arise from their behavior?</LI>
    <LI>Run the code. Can you observe a global pattern arising from the local behavior? If yes, why do you think this arises? Can you break the pattern by changing the random distributions? If no, why do you think there is no pattern?</LI>
</OL>

In [ ]:
%matplotlib inline
from swarming import *

# extend the agent class to take random actions:
class RandomAgent(Agent):
    
    def set_command(self):
        
        # draw velocity and rate from a uniform distribution:
        self.command_v = np.random.rand(1)[0]
        self.command_rate = 0.1 * np.pi * (np.random.rand(1)[0] * 2 - 1)

# create environment and agents:
env = Environment(100, 100)
for i in range(10):
    a = RandomAgent(env)
    env.add_agent(a)


# run the sim for a number of time steps and draw it:
dt = 0.1
n_time_steps = 10000

for t in range(n_time_steps):
    
    env.update_agents(dt)
    
    if(np.mod(t, 100) == 0):
        env.draw()

KeyboardInterrupt: 

## Flocking

We will now make a next step, and create a swarm that attempts to behave like a flock of birds. It is often said that the complex behavior of such flocks can be adequately modeled by means of a few simple rules, e.g., [3-5]. To put this to the test, we will implement the following famous flocking rules:

<OL>
    <LI>Separation: Avoid collisions with other swarm members</LI>
    <LI>Cohesion: Don't get separated from the flock</LI>
    <LI>Alignment: Align your velocity with that of your neighbors</LI>
</OL>

In order to implement these behaviors, we will make use of the function in the Agent class called `sense_nearest_neighbors`. It has two parameters, `k` which is the maximum number of neighbors considered, and `max_range` outside of which other swarm members are not observed. The observation model is omnidirectional and allows an agent to determine the velocity of the other agents (see the alignment rule, which needs this). The outputs of this function consists of: `neighbors`, the indices of the agents as ranked in the environment, `delta_pos` which contains per neighbor the relative position in body coordinates, and `delta_v` which gives the relative velocity in body coordinates. Please note that the relative velocity means that if the agent itself is moving forward with 1 m/s ($\mathbf{v_a} = [1, 0]^T$) and the other agent is hovering ($\mathbf{v_o} = [0,0]^T$), that this gives a relative velocity of that other agent of $\Delta \mathbf{v} = \mathbf{v_o} - \mathbf{v_a} = [-1, 0]^T$.

In order to understand the relative coordinates, please have a look at the coordinate frames below, where $X$ and $Y$ indicate the inertial frame, and $x_B$ and $y_B$ the body frame. Also please note that the agents have a maximal range, beyond which they will not detect other agents (dashed circles). Hence, if there were only these two agents, they would not perceive each other and receive empty lists from the `sense_nearest_neighbors` function. Furthermore, as can be seen in the figure, a more positive heading makes the drone rotate counter-clockwise. 

<IMG SRC="coordinate_systems.png" WIDTH="500"></IMG>


<FONT COLOR="red"><B>Exercise 2</B></FONT>

In this exercise, you are going to implement a simple flocking model. Below, you find a class MyFlockingAgent. It extends the Agent class, only replacing the `set_command` function. At the start of the function, the nearest neighbors are sensed. This gives a list of agent indices (as stored in the environment), and - more importantly - a list of relative positions and relative velocities in the body frame. In the code, you need to set a desired velocity vector `desired_v`. Our controller actually only uses the body $y$-coordinate of this vector, which it uses to set the rate. The velocity is kept constant in our example. Finally, the agent calls the function `avoid_wall`, which overrides the commanded rate to avoid a wall when too close. 

<OL>
    <LI>Implement the separation behavior. Make it a zero vector if all neighbors are outside of an 'avoidance radius' $R$ with respect to agent $a$. Sum the relative position vectors for all neighboring agents $n$ that are inside the radius, normalize it, and then fly into the other direction: $\mathbf{v}_{S} = -\sum_{n \in N_R} (\Delta \mathbf{p}_{n}) / \left\Vert \sum_{n \in N_R} (\Delta \mathbf{p}_{n}) \right \Vert$.</LI>
    <LI>Implement the cohesion behavior. Make it a desired vector that points towards the center of all agents in sight. Don't forget the agent itself ; ) The formula: $\mathbf{v}_{C} = \frac{\sum_{n \in N} (\Delta \mathbf{p}_{n}) / (|N|+1)}{\left\Vert \sum_{n \in N} (\Delta \mathbf{p}_{n}) / (|N|+1) \right\Vert}$.  </LI>
    <LI>Implement the alignment behavior. Make it a desired vector that changes the velocity in the direction of where the other agents are going. Note that due to the constant velocity model, the agent itself is always travelling at its cruise speed straight ahead. Add the average relative velocity to this vector: $\mathbf{v}_{A} = \frac{\mathbf{v}_a + \sum_{n \in N} (\Delta \mathbf{v}_n) / |N|}{\left\Vert \mathbf{v}_a + \sum_{n \in N} (\Delta \mathbf{v}_n) / |N| \right\Vert}$.</LI>
    <LI>You now have three desired normalized direction vectors, $\mathbf{v}_{S}, \mathbf{v}_{C}, \mathbf{v}_{A}$. The final desired velocity vector $\mathbf{v}_a^*$ (`desired_v` in the code) is a weighted sum of these vectors, which is subsequently scaled to the constant commanded velocity $c$: $\mathbf{v}_a^* = c \frac{w_A \mathbf{v}_A + w_S \mathbf{v}_S + w_C \mathbf{v}_C}{\left \Vert w_A \mathbf{v}_A + w_S \mathbf{v}_S + w_C \mathbf{v}_C \right \Vert}$</LI>
    <LI>How can you check that your code is working?</LI>
    <LI>Can you find values for your parameters (avoidance radius, max range, number of neighbors, weights, etc.) that lead to naturally looking flocking behavior?</LI>
</OL>

In [ ]:
%matplotlib inline
from swarming import *

# extend the agent class to take random actions:
class MyFlockingAgent(Agent):
    
    def set_command(self):
        
        # sense the neighbors:
        [neighbors, delta_pos, delta_v] = self.sense_nearest_neighbors(k=5, max_range = 20.0)
        n_neighbors = len(neighbors)
        
        if(n_neighbors == 0):
            # no neighbors, keep going in the same direction:
            self.command_rate = 0.0
            return
        
        # parameters of the method:
        command_velocity = 0.25
        rate_gain = 0.10
        # ...
        
        # initializing variables:
        # ...
        
        for n in range(n_neighbors):
            # ...
                
        # set the new commanded velocity and rate:
        self.command_v = command_velocity
        self.command_rate = -rate_gain * desired_v[1][0]
        
        # when close to a wall, the command rate is overwritten:
        self.avoid_wall()
        

# create environment and agents:
env = Environment(100, 100)
for i in range(10):
    a = MyFlockingAgent(env)
    env.add_agent(a)

# run the sim for a number of time steps and draw it:
dt = 0.1
n_time_steps = 10000

for t in range(n_time_steps):
    
    env.update_agents(dt)
   
    if(np.mod(t, 100) == 0):
        env.draw()


## Answers

<FONT COLOR="red"><B>Exercise 1</B></FONT>
<OL>
    <LI>There is no good answer for what to expect. The most common expectation is probably that no pattern will arise, as the agents just act randomly.</LI>
    <LI>The agents tend to accumulate at the borders of the environment. The state update equations in the agent do not allow it to cross those borders. This happens because the heading changes much less than the velocity. If you change `self.command_rate = 0.1 * np.pi * (np.random.rand(1) * 2 - 1)` to `self.command_rate = 1.0 * np.pi * (np.random.rand(1) * 2 - 1)` agents will also leave the border again. The interaction that leads to this behavior is the combination of the environment and the random agent - although arguably this interaction is not so complex.</LI>
</OL>


<FONT COLOR="red"><B>Exercise 2</B></FONT>
<OL>
    <LI>See the code in the block below.</LI>
    <LI>See the code in the block below.</LI>
    <LI>See the code in the block below.</LI>
    <LI>See the code in the block below.</LI>
    <LI>One option is to unit test the different behaviors. Set only one weight to be nonzero, like `w_separation = 20.0` while the others are zero. Switch off wall avoidance by commenting out `self.avoid_wall()`. Also set the `max_range` and `avoidance_radius` to high numbers like `1000.0`. The agents will now be able to always see other agents and will want to avoid them. The result should be that all agents accumulate at the borders. Doing the same for cohesion should lead to all agents being close(r). Only aligning will have all agents move in the same direction, irrespective of their relative positions.</LI>
    <LI>It is difficult to evaluate when a setting is successful in leading to 'natural behavior'. The settings below give a nice flocking result, although it is definitely possible to get more natural results. Perhaps different rules will be necessary then, as explored in other studies.</LI>
</OL>


In [ ]:
%matplotlib inline
from swarming import *

# extend the agent class to take random actions:
class FlockingAgent(Agent):
    
    def set_command(self):
        
        # sense the neighbors:
        [neighbors, delta_pos, delta_v] = self.sense_nearest_neighbors(k=5, max_range = 20.0)
        n_neighbors = len(neighbors)
        
        if(n_neighbors == 0):
            # no neighbors, keep going in the same direction:
            self.command_rate = 0.0
            self.avoid_wall()
            return
        
        # parameters of the method:
        avoidance_radius = 10.0
        w_separation = 50.0
        w_cohesion = 10.0
        w_alignment = 20.0
        command_velocity = 0.25
        rate_gain = 0.10
        
        # initializing variables:
        avoidance_vector = np.zeros([2,1])
        avoidance = False
        flock_centroid = np.zeros([2,1])
        delta_v_align = np.zeros([2,1])
        
        for n in range(n_neighbors):
            d_pos = delta_pos[n]
            distance = np.sqrt(d_pos[0]*d_pos[0] + d_pos[1]*d_pos[1])
            
            # separation:
            if distance < avoidance_radius:
                avoidance = True
                avoidance_vector -= d_pos / np.linalg.norm(d_pos)
        
            # cohesion:
            flock_centroid += d_pos / np.linalg.norm(d_pos)
            
            # alignment:
            d_vel = delta_v[n]
            delta_v_align += d_vel
        
        if(avoidance):
            # normalize the avoidance vector:
            avoidance_vector /= np.linalg.norm(avoidance_vector)
        else:
            avoidance_vector = np.zeros([2,1])
        
        # cohesion vector:
        flock_centroid /= (n_neighbors + 1) # +1 because the agent itself counts for the centroid
        flock_centroid /= np.linalg.norm(flock_centroid)
        
        # alignment:
        v_body = np.zeros([2,1])
        v_body[0] = self.v
        v_body[1] = 0
        v_alignment = v_body + delta_v_align / n_neighbors
        
        desired_v = w_separation * avoidance_vector + w_cohesion * flock_centroid + w_alignment * v_alignment 
        norm_dv = np.linalg.norm(desired_v)
        if norm_dv > 0.0001:
            desired_v /= norm_dv
        desired_v *= command_velocity
        
        # set the new commanded velocity and rate:
        self.command_v = command_velocity
        self.command_rate = -rate_gain * desired_v[1][0]
        
        # when close to a wall, the command rate is overwritten:
        self.avoid_wall()
        

# create environment and agents:
env = Environment(100, 100)
for i in range(10):
    a = FlockingAgent(env)
    env.add_agent(a)

# run the sim for a number of time steps and draw it:
dt = 0.1
n_time_steps = 10000

for t in range(n_time_steps):
    
    env.update_agents(dt)
   
    if(np.mod(t, 100) == 0):
        env.draw()


## References

[1] Şahin, E. (2004, July). Swarm robotics: From sources of inspiration to domains of application. In International workshop on swarm robotics (pp. 10-20). Springer, Berlin, Heidelberg.

[2] Brambilla, M., Ferrante, E., Birattari, M., & Dorigo, M. (2013). Swarm robotics: a review from the swarm engineering perspective. Swarm Intelligence, 7(1), 1-41.

[3] Reynolds, C. W. (1987, August). Flocks, herds and schools: A distributed behavioral model. In Proceedings of the 14th annual conference on Computer graphics and interactive techniques (pp. 25-34).

[4] Hildenbrandt, H., Carere, C., & Hemelrijk, C. K. (2010). Self-organized aerial displays of thousands of starlings: a model. Behavioral Ecology, 21(6), 1349-1359.

[5] Vásárhelyi, G., Virágh, C., Somorjai, G., Nepusz, T., Eiben, A. E., & Vicsek, T. (2018). Optimized flocking of autonomous drones in confined environments. Science Robotics, 3(20).